In [ ]:
import os
import cv2
import pickle
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import mannwhitneyu


def load_dataset():
    """
    Loads the datasets encoded in .pkl files and returns its decoded form.

    Returns
    -------
    list
        A list of n-dimensional arrays representing the subjects samples that will be used to \\
        train the drunkenness classification model.
    ndarray
        A n-dimensional array representing the samples labels.
    """

    print("Loading Sober-Drunk Face Dataset, from Patras University")
    
    # Defining the sample and label sets filenames
    sets = [
        "Insert the x_balanced.pkl file path here",
        "Insert the y_balanced.pkl file path here"   
    ]
    
    # Defining an empty list for storing the decoded dataset
    loaded_datasets = []
 
    # Iterating over the dataset files
    for set_ in sets:
        # Opening the .pkl file in read mode
        with open(set_, 'rb') as file:
            # Appending the decoded dataset to the dataset list
            loaded_datasets.append(pickle.load(file))
    
    # Unpacking the dataset list into individual subsets
    x, y = loaded_datasets
    
    # Converting the label list to the n-dimensional array format
    y_arr= np.array(y)
    
    # Printing the dataset length for sanity check
    print("\nSamples total: {0}".format(len(x)))
    
    # Slicing the frame sequences of each subject for selecting the
    # frames sampled at each 5 Hz
    x = x[::5]
    y_arr = y_arr[::5]
    
    # Printing the dataset length after slicing for sanity check
    print("\nSamples total after slicing: {0}".format(len(x)))
    
    # Returning the samples set and its respective labels
    return x, y_arr

        
def normalize_img(img_arr):
    """
    Receives a 16-bit frame and normalizes it to the 8-bit format with pixel values ranging from 0 to 255.

    Parameters
    ----------
    img_array : ndarray
        The frame to be normalized.

    Returns
    -------
    ndarray
        A 8-bit n-dimensional array with values ranging from 0 to 255.
    """

    normalized_img = np.zeros((128, 160))
    normalized_img = cv2.normalize(img_arr, normalized_img, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)
    normalized_img = cv2.convertScaleAbs(normalized_img)
    
    return normalized_img


def display_and_save(sample, roi, right_side, left_side, save_plot=False, filename=None):
    """
    Displays the face regions of interest considered for pixel behavior analysis.

    Parameters
    ----------
    sample : ndarray
        A keyframe in grayscale.
    roi : ndarray
        The image obtained after extracting the face region from the background.
    right_side : ndarray
        A sliced ndarray representing the roi pixels from right to center.
    left_side : ndarray
        A sliced ndarray representing the roi pixels from center to left.
    save_plot : bool
        A flag indicating whether to save the plot.
    filename : str
        A string indicating the filename to save the plot.
    """

    # Displaying and saving the subject sample
    plt.figure()
    #plt.title("Sample")
    plt.imshow(sample, cmap='gray')
    plt.xticks([])
    plt.yticks([])

    if save_plot == True:
        plt.savefig("{0}_sample.pdf".format(filename), dpi=600, bbox_inches='tight', pad_inches=0.001)
    
    # Displaying and saving the face region of interest
    plt.figure()
    #plt.title("Region of Interest")
    plt.imshow(roi, cmap='gray')
    plt.xticks([])
    plt.yticks([])

    if save_plot == True:
        plt.savefig("{0}_roi.pdf".format(filename), dpi=600, bbox_inches='tight', pad_inches=0.001)

    # Displaying and saving the face's right side
    plt.figure()
    #plt.title("Face's right side")
    plt.imshow(right_side, cmap='gray')
    plt.xticks([])
    plt.yticks([])

    if save_plot == True:
        plt.savefig("{0}_right-side.pdf".format(filename), dpi=600, bbox_inches='tight', pad_inches=0.001)

    # Displaying and saving the face's left side
    plt.figure()
    #plt.title("Face's left side")
    plt.imshow(left_side, cmap='gray')
    plt.xticks([])
    plt.yticks([])

    if save_plot == True:
        plt.savefig("{0}_left-side.pdf".format(filename), dpi=600, bbox_inches='tight', pad_inches=0.001)

    plt.show()


def update_center_coordinates(cX, cY, idx):
    """
    Updates the ellipse mask center coordinates based on the given index.

    Parameters
    ----------
    cX : int
        The X coordinate.
    cY : int 
        The Y coordinate.
    idx : int
        The index to select the specific coordinate pair.

    Returns
    -------
    tuple
        The updated coordinate pair based on the index.
    """

    # Defining a list of possible center coordinates based on given (cX, cY)
    center_coordinates = [
        (cX, cY),      # subject 1, acquisition period 1
        (cX+3, cY-3),  # subject 1, acquisition period 4
        (cX-2, cY-4),  # subject 2, acquisition period 1
        (cX+3, cY),    # subject 2, acquisition period 4
        (cX+1, cY-1),  # subject 3, acquisition period 1
        (cX+1, cY-1),  # subject 3, acquisition period 4
        (cX-1, cY),    # subject 4, acquisition period 1
        (cX+2, cY+2),  # subject 4, acquisition period 4
        (cX-4, cY+4),  # subject 5, acquisition period 1
        (cX-2, cY),    # subject 5, acquisition period 4
        (cX-2, cY),    # subject 6, acquisition period 1
        (cX-2, cY-2),  # subject 6, acquisition period 4
        (cX-2, cY-2),  # subject 7, acquisition period 1
        (cX-2, cY+6),  # subject 7, acquisition period 4
        (cX, cY+4),    # subject 8, acquisition period 1
        (cX+2, cY+6),  # subject 8, acquisition period 4
        (cX+1, cY+6),  # subject 9, acquisition period 1
        (cX-1, cY+6),  # subject 9, acquisition period 4
        (cX-5, cY+5),  # subject 10, acquisition period 1
        (cX+2, cY+5),  # subject 10, acquisition period 4
        (cX, cY),      # subject 11, acquisition period 1
        (cX, cY),      # subject 11, acquisition period 4
        (cX+1, cY+4),  # subject 12, acquisition period 1
        (cX+2, cY+4),  # subject 12, acquisition period 4
        (cX-2, cY+2),  # subject 13, acquisition period 1
        (cX-2, cY+2),  # subject 13, acquisition period 4
        (cX-2, cY),    # subject 14, acquisition period 1
        (cX, cY+4),    # subject 14, acquisition period 4
        (cX-2, cY),    # subject 15, acquisition period 1
        (cX-2, cY),    # subject 15, acquisition period 4
        (cX, cY+4),    # subject 16, acquisition period 1
        (cX, cY+8),    # subject 16, acquisition period 4
        (cX-5, cY+5),  # subject 17, acquisition period 1
        (cX-1, cY+5),  # subject 17, acquisition period 4
        (cX-1, cY+1),  # subject 18, acquisition period 1
        (cX-4, cY-3),  # subject 18, acquisition period 4
        (cX-1, cY+4),  # subject 19, acquisition period 1
        (cX, cY),      # subject 19, acquisition period 4
        (cX-3, cY+2),  # subject 21, acquisition period 1
        (cX-1, cY+1),  # subject 21, acquisition period 4
        (cX, cY+1),    # subject 22, acquisition period 1
        (cX+2, cY+1),  # subject 22, acquisition period 4
        (cX-2, cY+10), # subject 23, acquisition period 1
        (cX-2, cY+6),  # subject 23, acquisition period 4
        (cX, cY+4),    # subject 24, acquisition period 1
        (cX+3, cY),    # subject 24, acquisition period 4
        (cX, cY+6),    # subject 25, acquisition period 1
        (cX+2, cY+6),  # subject 25, acquisition period 4
        (cX-1, cY+1),  # subject 26, acquisition period 1
        (cX, cY+4),    # subject 26, acquisition period 4
        (cX-4, cY+4),  # subject 27, acquisition period 1
        (cX-4, cY),    # subject 27, acquisition period 4
        (cX+1, cY+8),  # subject 28, acquisition period 1
        (cX+2, cY+6),  # subject 28, acquisition period 4
        (cX-2, cY+8),  # subject 29, acquisition period 1
        (cX-1, cY+4),  # subject 29, acquisition period 4
        (cX-2, cY+4),  # subject 30, acquisition period 1
        (cX+1, cY+5),  # subject 30, acquisition period 4
        (cX+1, cY),    # subject 31, acquisition period 1
        (cX+1, cY),    # subject 31, acquisition period 4
        (cX+5, cY+5),  # subject 32, acquisition period 1
        (cX+3, cY),    # subject 32, acquisition period 4
        (cX+2, cY),    # subject 33, acquisition period 1
        (cX+4, cY),    # subject 33, acquisition period 4
        (cX, cY),      # subject 34, acquisition period 1
        (cX, cY),      # subject 34, acquisition period 4
        (cX, cY),      # subject 35, acquisition period 1
        (cX+1, cY-2),  # subject 35, acquisition period 4
        (cX+3, cY+3),  # subject 36, acquisition period 1
        (cX-1, cY+1),  # subject 36, acquisition period 4
        (cX+2, cY),    # subject 37, acquisition period 1
        (cX, cY),      # subject 37, acquisition period 4
        (cX-1, cY+2),  # subject 38, acquisition period 1
        (cX-1, cY-2),  # subject 38, acquisition period 4
        (cX+4, cY+4),  # subject 39, acquisition period 1
        (cX+2, cY+4),  # subject 39, acquisition period 4
        (cX+2, cY-2),  # subject 40, acquisition period 1
        (cX, cY-10),   # subject 40, acquisition period 4
        (cX+2, cY-8),  # subject 41, acquisition period 1
        (cX, cY+4)     # subject 41, acquisition period 4
    ]
    
    # Returning the coordinate pair corresponding to the provided index (subject)
    return center_coordinates[idx]


def get_axes_length(idx):
    """
    Retrieves the ellipse mask axes based on the given index.

    Parameters
    ----------
    idx : int
        The index to select the specific axes length pair.

    Returns
    -------
    tuple
        The length of the X and Y axes based on the index.
    """
    
    # Defining a list of possible axes lengths
    axes_length = [
        (40, 60),      # subject 1, acquisition period 1
        (34, 54),      # subject 1, acquisition period 4
        (34, 54),      # subject 2, acquisition period 1
        (36, 60),      # subject 2, acquisition period 4
        (35, 55),      # subject 3, acquisition period 1
        (33, 53),      # subject 3, acquisition period 4
        (40, 60),      # subject 4, acquisition period 1
        (35, 55),      # subject 4, acquisition period 4
        (40, 60),      # subject 5, acquisition period 1
        (40, 60),      # subject 5, acquisition period 4
        (40, 60),      # subject 6, acquisition period 1
        (35, 50),      # subject 6, acquisition period 4
        (35, 50),      # subject 7, acquisition period 1
        (38, 58),      # subject 7, acquisition period 4
        (42, 60),      # subject 8, acquisition period 1
        (40, 57),      # subject 8, acquisition period 4
        (37, 60),      # subject 9, acquisition period 1
        (38, 60),      # subject 9, acquisition period 4
        (40, 60),      # subject 10, acquisition period 1
        (42, 60),      # subject 10, acquisition period 4
        (40, 60),      # subject 11, acquisition period 1
        (40, 60),      # subject 11, acquisition period 4
        (45, 60),      # subject 12, acquisition period 1
        (42, 60),      # subject 12, acquisition period 4
        (38, 58),      # subject 13, acquisition period 1
        (38, 58),      # subject 13, acquisition period 4
        (40, 60),      # subject 14, acquisition period 1
        (45, 65),      # subject 14, acquisition period 4
        (37, 57),      # subject 15, acquisition period 1
        (33, 50),      # subject 15, acquisition period 4
        (36, 50),      # subject 16, acquisition period 1
        (40, 57),      # subject 16, acquisition period 4
        (42, 60),      # subject 17, acquisition period 1
        (40, 60),      # subject 17, acquisition period 4
        (32, 52),      # subject 18, acquisition period 1
        (32, 50),      # subject 18, acquisition period 4
        (37, 57),      # subject 19, acquisition period 1
        (40, 60),      # subject 19, acquisition period 4
        (37, 57),      # subject 21, acquisition period 1
        (35, 50),      # subject 21, acquisition period 4
        (35, 55),      # subject 22, acquisition period 1
        (35, 55),      # subject 22, acquisition period 4
        (34, 50),      # subject 23, acquisition period 1
        (40, 60),      # subject 23, acquisition period 4
        (37, 57),      # subject 24, acquisition period 1
        (39, 57),      # subject 24, acquisition period 4
        (37, 57),      # subject 25, acquisition period 1
        (40, 60),      # subject 25, acquisition period 4
        (34, 48),      # subject 26, acquisition period 1
        (42, 58),      # subject 26, acquisition period 4
        (35, 55),      # subject 27, acquisition period 1
        (38, 58),      # subject 27, acquisition period 4
        (43, 60),      # subject 28, acquisition period 1
        (40, 60),      # subject 28, acquisition period 4
        (45, 60),      # subject 29, acquisition period 1
        (43, 60),      # subject 29, acquisition period 4
        (40, 60),      # subject 30, acquisition period 1
        (40, 60),      # subject 30, acquisition period 4
        (37, 57),      # subject 31, acquisition period 1
        (37, 57),      # subject 31, acquisition period 4
        (40, 60),      # subject 32, acquisition period 1
        (46, 66),      # subject 32, acquisition period 4
        (40, 60),      # subject 33, acquisition period 1
        (35, 55),      # subject 33, acquisition period 4
        (37, 57),      # subject 34, acquisition period 1
        (35, 55),      # subject 34, acquisition period 4
        (40, 60),      # subject 35, acquisition period 1
        (37, 56),      # subject 35, acquisition period 4
        (35, 55),      # subject 36, acquisition period 1
        (34, 50),      # subject 36, acquisition period 4
        (36, 60),      # subject 37, acquisition period 1
        (38, 60),      # subject 37, acquisition period 4
        (40, 60),      # subject 38, acquisition period 1
        (37, 57),      # subject 38, acquisition period 4
        (40, 60),      # subject 39, acquisition period 1
        (40, 60),      # subject 39, acquisition period 4
        (33, 53),      # subject 40, acquisition period 1
        (30, 45),      # subject 40, acquisition period 4
        (34, 48),      # subject 41, acquisition period 1
        (36, 52)       # subject 41, acquisition period 4
    ]
    
    # Returning the axes length pair corresponding to the provided index
    return axes_length[idx]
    
    
def find_centroid(image, idx):
    """
    Calculates the image moments and returns its central points along the face region of interest.

    Parameters
    ----------
    image : ndarray
        A keyframe in grayscale.

    Returns
    -------
    cX : int
        The image horizontal axis central point index.
    cY : int
        The image vertical axis central point index.
    roi : ndarray
        A n-dimensional array representing the face region of interest extracted by an ellipse mask.
    """

    # Defining an image copy for the binarization process
    roi = image.copy()

    # Defining the region of interest n-dimensional array
    mask = np.zeros_like(roi)

    # Performing the image binarization
    ret, thresh = cv2.threshold(roi, 0, 255, cv2.THRESH_OTSU)
    
    # Calculating the binary image moments
    M = cv2.moments(thresh)
    
    # Calculating the centroid coordinates
    if M["m00"] != 0:
        cX = int(M["m10"] / M["m00"]) # horizontal axis
        cY = int(M["m01"] / M["m00"]) # vertical axis
    else:
        cX, cY = 0, 0
    
    # Defining the ellipse mask central points
    center_coordinates = update_center_coordinates(cX, cY, idx)

    # Defining the ellipse mask width and height 
    axesLength = get_axes_length(idx)

    # Defining the ellipse mask angle
    angle = 0
    startAngle = 0
    endAngle = 360   

    # Defining the ellipse color and line thickness
    color = (255, 255, 255)
    thickness = -1
     
    # Drawing a white ellipse in a black background
    mask = cv2.ellipse(mask, center_coordinates, axesLength, angle, 
                       startAngle, endAngle, color, thickness)
    
    # Performing a bit-wise AND operation between the ellipse mask and the given keyframe 
    roi = np.bitwise_and(roi, mask)
    
    # Finding the indices of non-zero elements in the region of interest (roi)
    x, y = np.nonzero(roi)
    # Determining the minimum and maximum x-coordinates of the non-zero elements
    xl, xr = x.min(), x.max()
    # Determining the minimum and maximum y-coordinates of the non-zero elements
    yl, yr = y.min(), y.max()
    # Cropping the roi to only include the non-zero region
    roi = roi[xl:xr+1, yl:yr+1]
    
    # Performing the roi binarization
    ret, thresh = cv2.threshold(roi, 0, 255, cv2.THRESH_OTSU)
    
    # Calculating the roi image moments
    M = cv2.moments(thresh)
    
    # Calculating the roi centroid coordinates
    if M["m00"] != 0:
        cX = int(M["m10"] / M["m00"]) # horizontal axis
        cY = int(M["m01"] / M["m00"]) # vertical axis
    else:
        cX, cY = 0, 0
    
    # Returning the keyframe centroid points and its region of interest
    return cX, cY, roi


def calculate_statistics(x):
    """
    Calculates the pixel average on the right and left sides of a subject's face.

    Parameters
    ----------
    x : ndarray
        A set of keyframes obtained in the 1st or 4th acquisition period.
    
    Returns
    -------
    statistics_sober : dict
        A dictionary containing the pixel averages for the sober state.
    statistics_drunk : dict
        A dictionary containing the pixel averages for the drunk state.
    """

    # Defining indexes and counters
    c = 0
    frame_list = range(0, 50, 5)
    period = 1
    frame_idx = 0
    subject = 1

    # Defining empty lists to store pixel averages for the roi's right and left sides
    pixel_avg_r = []
    pixel_avg_l = []

    # Defining dictionaries to store statistics for each subject
    statistics_sober = {"subject_1": {}}
    statistics_drunk = {"subject_1": {}}

    # Iterating over the dataset samples
    for sample in x:
        print("\nSubject {0}, frame {1}, acquisition period {2}".format(subject, frame_list[c], period))

        # Normalizing the sample data type and pixel scale
        normalized_sample = normalize_img(sample)
        # Converting the normalized sample from RGB to Grayscale
        normalized_sample = cv2.cvtColor(normalized_sample, cv2.COLOR_RGB2GRAY)

        # Finding the keyframe centroid coordinates and its region of interest
        cX, cY, roi = find_centroid(normalized_sample, frame_idx)

        # Normalizing the pixels scale [0-1]
        roi = roi / 255.0

        # Slicing the image array to find the face's right side
        right_side = roi[:, :cX]
    
        # Slicing the image array to find the face's left side
        left_side = roi[:, cX:]
    
        # Selecting the valid pixels for statistics calculation
        right_side_valid_pixels = right_side[right_side > 0.0] # only the pixels inside the ellipse mask
        left_side_valid_pixels = left_side[left_side > 0.0]    # only the pixels inside the ellipse mask
    
        # Calculating the average of pixels on the right side of a subject's face
        right_mean = np.mean(right_side_valid_pixels)
    
        # Calculating the average of pixels on the left side of a subject's face
        left_mean = np.mean(left_side_valid_pixels)

        # Appending the face's right side pixel average to the list
        pixel_avg_r.append(np.mean(right_side_valid_pixels))
        # Appending the face's left side pixel average to the list
        pixel_avg_l.append(np.mean(left_side_valid_pixels))
    
        print("Pixel average from face's right side: {0:0.2f}".format(right_mean))
        print("Pixel average from face's left side: {0:0.2f}".format(left_mean))

        # Checking if the sample corresponds the sober or drunk state
        if period == 1:
            # Storing the pixel averages in the statistics dictionary for sober subjects
            statistics_sober["subject_{0}".format(subject)]["right_side_avg"] = pixel_avg_r
            statistics_sober["subject_{0}".format(subject)]["left_side_avg"] = pixel_avg_l
        elif period == 4:
            # Store the pixel averages in the statistics dictionary for drunk subjects
            statistics_drunk["subject_{0}".format(subject)]["right_side_avg"] = pixel_avg_r
            statistics_drunk["subject_{0}".format(subject)]["left_side_avg"] = pixel_avg_l
    
        # Defining the sample file path
        filename = "subject-{0}_acquisition_period-{1}_frame-{2}".format(subject, period, frame_list[c])

        # Displaying the given subject's keyframe region of interest and its face's right and left sides
        #if subject == 38 and frame_list[c] == 0 and period == 4:
        #    display_and_save(sample=normalized_sample, roi=roi, right_side=right_side, left_side=left_side, save_plot=True, filename=filename)
        #else:
        #    display_and_save(roi=roi, right_side=right_side, left_side=left_side)
    
        # Updating indexes and counters
        c += 1
    
        if c >= 10:
            c = 0
            period +=3
            frame_idx += 1
            pixel_avg_r = []
            pixel_avg_l = []
    
        if period > 4:
            period = 1
            subject +=1
        
            if subject == 20:
                subject += 1
        
            if subject <= 41:
                statistics_sober["subject_{0}".format(subject)] = {}
                statistics_drunk["subject_{0}".format(subject)] = {}
    
    # Returning the pixel averages of each side of the face for every subject in both states
    return statistics_sober, statistics_drunk


# Loading the Sober-Drunk Dataset samples
x, y = load_dataset()

# Calculating statics from a subject facial image
statistics_sober, statistics_drunk = calculate_statistics(x)


In [ ]:
def extract_values(statistics):
    """
    Extracts and flattens the pixel averages from the right and left sides of faces.

    Parameters
    ----------
    statistics : dict
        A dictionary containing the pixel averages for subjects.

    Returns
    -------
    pixel_avg_r : list
        A flattened list of pixel averages from the right side of faces.
    pixel_avg_l : list
        A flattened list of pixel averages from the left side of faces.
    """

    # Defining empty lists to store the pixel averages from the roi's right and left sides
    pixel_avg_r = []
    pixel_avg_l = []

    # Iterate through each subject and their statistics
    for subject, info in statistics.items():    
        for key in info:
            if key == "right_side_avg":
                # Appending the right side averages to the list
                pixel_avg_r.append(info[key])
            else:
                # Appending the left side averages to the list
                pixel_avg_l.append(info[key])
    
    # Flattening the list of pixel averages from face's right side
    pixel_avg_r = sum(pixel_avg_r, [])

    # Flattening the list of pixel averages from face's left side
    pixel_avg_l = sum(pixel_avg_l, [])
    
    # Returning the flattened lists of pixel averages
    return pixel_avg_r, pixel_avg_l


def hypothesis_test_all_frames(sober_pixel_avg_r, sober_pixel_avg_l, drunk_pixel_avg_r, drunk_pixel_avg_l):
    """
    Performs a Mann-Whitney hypothesis test on pixel averages from the right and left sides of faces in sober and drunk states.

    Parameters
    ----------
    sober_pixel_avg_r : list
        List of pixel averages from the right side of faces in the sober state.
    sober_pixel_avg_l : list
        List of pixel averages from the left side of faces in the sober state.
    drunk_pixel_avg_r : list
        List of pixel averages from the right side of faces in the drunk state.
    drunk_pixel_avg_l : list
        List of pixel averages from the left side of faces in the drunk state.

    Returns
    -------
    None
    """

    # Running the Mann-Whitney hypothesis test
    print("=======================================================================================")

    print("\nHypothesis test regarding the face's right side thermal symmetry between the sober and drunk states")
    # A p-value less than alpha (0.05) rejects the Mann-Whitney test null hypothesis (that the distribution underlying 
    # sample x is the same as the distribution underlying sample y), indicating that the face's right side pixel average
    # distributions vary between the 1st and 4th acquisition periods.

    stat, p = mannwhitneyu(sober_pixel_avg_r, drunk_pixel_avg_r, alternative='two-sided')
    print("\nMann-Whitney p-value: {0}".format(p))

    print("\n=======================================================================================")

    print("\nHypothesis test regarding the face's left side thermal symmetry between the sober and drunk states")
    # A p-value less than alpha (0.05) rejects the Mann-Whitney test null hypothesis (that the distribution underlying 
    # sample x is the same as the distribution underlying sample y), indicating that the face's left side pixel average
    # distributions vary between the 1st and 4th acquisition periods.

    stat, p = mannwhitneyu(sober_pixel_avg_l, drunk_pixel_avg_l, alternative='two-sided')
    print("\nMann-Whitney p-value: {0}".format(p))

    print("\n=======================================================================================")


def hypothesis_test_per_frame(sober_pixel_avg_r, sober_pixel_avg_l, drunk_pixel_avg_r, drunk_pixel_avg_l):
    """
    Performs a Mann-Whitney hypothesis test on pixel averages from the right and left sides of faces in sober and drunk states for each frame.

    Parameters
    ----------
    sober_pixel_avg_r : list
        List of pixel averages from the right side of faces in the sober state.
    sober_pixel_avg_l : list
        List of pixel averages from the left side of faces in the sober state.
    drunk_pixel_avg_r : list
        List of pixel averages from the right side of faces in the drunk state.
    drunk_pixel_avg_l : list
        List of pixel averages from the left side of faces in the drunk state.

    Returns
    -------
    p_values_r : list
        List of p-values for the hypothesis test on the right side pixel averages.
    """
    
    # Defining thresholds
    frames = 10
    frame_interval = 10
    subjects = 40
    
    # Defining the counter and frame list indexes
    c = 0
    frame_list = range(0,50,5)
    
    # Defining an empty List to store p-values for the right side hypothesis tests
    p_values_r = []
    
    print("=======================================================================================")

    #Iterating over each subject
    for subject in range(1, subjects+1):
        # Iterating through the keyframes of each subject
        for frame in range(frames):
            # Updating the subject index      
            if subject == 20:
                print("\nSubject {0}, frame {1}".format(subject+1, frame_list[c]))
            else:
                print("\nSubject {0}, frame {1}".format(subject, frame_list[c]))
            
            # Running the Mann-Whitney hypothesis test
            print("\n=======================================================================================")

            print("\nHypothesis test regarding the face's right side thermal symmetry between the sober and drunk states")
            # A p-value less than alpha (0.05) rejects the Mann-Whitney test null hypothesis (that the distribution underlying 
            # sample x is the same as the distribution underlying sample y), indicating that the face's right side pixel average
            # distributions vary between the 1st and 4th acquisition periods.

            stat, p = mannwhitneyu(sober_pixel_avg_r[frame::frame_interval], drunk_pixel_avg_r[frame::frame_interval], alternative='two-sided')
            print("\nMann-Whitney p-value: {0}".format(p))
            
            # Appending the right side p-values to the list
            p_values_r.append(p)

            print("\n=======================================================================================")

            print("\nHypothesis test regarding the face's left side thermal symmetry between the sober and drunk states")
            # A p-value less than alpha (0.05) rejects the Mann-Whitney test null hypothesis (that the distribution underlying 
            # sample x is the same as the distribution underlying sample y), indicating that the face's left side pixel average
            # distributions vary between the 1st and 4th acquisition periods.

            stat, p = mannwhitneyu(sober_pixel_avg_l[frame::frame_interval], drunk_pixel_avg_l[frame::frame_interval], alternative='two-sided')
            print("\nMann-Whitney p-value: {0}".format(p))

            print("\n=======================================================================================")
            
            # Updating indexes and counters
            c += 1
    
            if c >= 10:
                c = 0
                
    return p_values_r

                
def thermal_dissimilarity_check(sober_pixel_avg_r, sober_pixel_avg_l, drunk_pixel_avg_r, drunk_pixel_avg_l):
    """
    Checks for thermal dissimilarities between sober and drunk states for each frame.

    Parameters
    ----------
    sober_pixel_avg_r : list
        List of pixel averages from the right side of faces in the sober state.
    sober_pixel_avg_l : list
        List of pixel averages from the left side of faces in the sober state.
    drunk_pixel_avg_r : list
        List of pixel averages from the right side of faces in the drunk state.
    drunk_pixel_avg_l : list
        List of pixel averages from the left side of faces in the drunk state.

    Returns
    -------
    None
    """
    
    # Defining thresholds
    frames = 10
    frame_interval = 10
    subjects = 41
    
    # Defining the counter and frame list indexes
    c = 0
    frame_list = range(0,50,5)

    print("=======================================================================================")

    # Iterating through the keyframes of each subject
    for frame in range(frames):
        print("\nFrame {0}".format(frame_list[c]))

        print("\n=======================================================================================")

        # Count and print comparisons of pixel averages between sober and drunk states and between sides of the face
        print("\ndrunk_pixel_avg_r != sober_pixel_avg_r: ", np.count_nonzero(np.asarray(drunk_pixel_avg_r[frame::frame_interval]) != np.asarray(sober_pixel_avg_r[frame::frame_interval])))
        print("drunk_pixel_avg_r > sober_pixel_avg_r: ", np.count_nonzero(np.asarray(drunk_pixel_avg_r[frame::frame_interval]) > np.asarray(sober_pixel_avg_r[frame::frame_interval])))
        print("drunk_pixel_avg_r < sober_pixel_avg_r: ", np.count_nonzero(np.asarray(drunk_pixel_avg_r[frame::frame_interval]) < np.asarray(sober_pixel_avg_r[frame::frame_interval])))

        print("\ndrunk_pixel_avg_l != sober_pixel_avg_l: ", np.count_nonzero(np.asarray(drunk_pixel_avg_l[frame::frame_interval]) != np.asarray(sober_pixel_avg_l[frame::frame_interval])))
        print("drunk_pixel_avg_l > sober_pixel_avg_l: ", np.count_nonzero(np.asarray(drunk_pixel_avg_l[frame::frame_interval]) > np.asarray(sober_pixel_avg_l[frame::frame_interval])))
        print("drunk_pixel_avg_l < sober_pixel_avg_l: ", np.count_nonzero(np.asarray(drunk_pixel_avg_l[frame::frame_interval]) < np.asarray(sober_pixel_avg_l[frame::frame_interval])))

        print("\ndrunk_pixel_avg_r != drunk_pixel_avg_l: ", np.count_nonzero(np.asarray(drunk_pixel_avg_r[frame::frame_interval]) != np.asarray(drunk_pixel_avg_l[frame::frame_interval])))
        print("drunk_pixel_avg_r > drunk_pixel_avg_l: ", np.count_nonzero(np.asarray(drunk_pixel_avg_r[frame::frame_interval]) > np.asarray(drunk_pixel_avg_l[frame::frame_interval])))
        print("drunk_pixel_avg_r < drunk_pixel_avg_l: ", np.count_nonzero(np.asarray(drunk_pixel_avg_r[frame::frame_interval]) < np.asarray(drunk_pixel_avg_l[frame::frame_interval])))

        print("\nsober_pixel_avg_r != sober_pixel_avg_l: ", np.count_nonzero(np.asarray(sober_pixel_avg_r[frame::frame_interval]) != np.asarray(sober_pixel_avg_l[frame::frame_interval])))
        print("sober_pixel_avg_r > sober_pixel_avg_l: ", np.count_nonzero(np.asarray(sober_pixel_avg_r[frame::frame_interval]) > np.asarray(sober_pixel_avg_l[frame::frame_interval])))
        print("sober_pixel_avg_r < sober_pixel_avg_l: ", np.count_nonzero(np.asarray(sober_pixel_avg_r[frame::frame_interval]) < np.asarray(sober_pixel_avg_l[frame::frame_interval])))

        print("\n=======================================================================================")
    
        # Updating indexes and counters
        c += 1
    
        if c >= 10:
            c = 0


# Extracting pixel averages for the right and left sides from the sober and drunk statistics
sober_pixel_avg_r, sober_pixel_avg_l = extract_values(statistics_sober)
drunk_pixel_avg_r, drunk_pixel_avg_l = extract_values(statistics_drunk)

# Performing a hypothesis test on all frames to compare sober and drunk pixel averages
hypothesis_test_all_frames(sober_pixel_avg_r, sober_pixel_avg_l, drunk_pixel_avg_r, drunk_pixel_avg_l)

In [ ]:
# Checking for thermal dissimilarities between sober and drunk states for each frame
thermal_dissimilarity_check(sober_pixel_avg_r, sober_pixel_avg_l, drunk_pixel_avg_r, drunk_pixel_avg_l)

In [ ]:
# Performing a hypothesis test per frame to compare sober and drunk pixel averages
p_values_r =  hypothesis_test_per_frame(sober_pixel_avg_r, sober_pixel_avg_l, drunk_pixel_avg_r, drunk_pixel_avg_l)

In [ ]:
# Displaying and saving the right side p-values probabilistic distribution between all analyzed frames

# Creating a new figure with a specified size
plt.figure(figsize=(10, 6))

# Creating a horizontal violin plot of the p-values
violin_parts = plt.violinplot(p_values_r, vert=False)
plt.xlim(left=0.00001, right=0.00016)  # Set the x-axis limits
plt.yticks(np.arange(0.75, 1.3, 0.1))  # Set the y-axis ticks

# Adding horizontal lines at the minimum and maximum y-values
plt.hlines(y=[0.75, 1.25], xmin=0, xmax=0.00016, colors='#23404b', linestyles='-', lw=0.8, alpha=0.3)

# Removing the top and right spines from the plot
sns.despine()

# Setting the plot's title, x-axis and y-axis labels
plt.title('Mann-Whitney p-values distribution analysis')
plt.xlabel("p-values from the face's right side")
plt.ylabel('Density')

# Customizing the violin plot's appearance
for part in violin_parts['bodies']:
    part.set_facecolor('#00FA9A')
    part.set_alpha(0.5)

violin_parts['cmaxes'].set_color('#23404b')
violin_parts['cmins'].set_color('#23404b')
violin_parts['cbars'].set_color('#23404b')

# Adding text annotations for the minimum and maximum p-values
plt.text(min(p_values_r), 0.85, str(np.round(min(p_values_r), 5)), ha='center', color='#23404b', alpha=1)
plt.text(max(p_values_r), 0.85, str(np.round(max(p_values_r), 5)), ha='center', color='#23404b', alpha=1)

# Saving the plot as a PDF file
plt.savefig("p-values_violin-plot.pdf", dpi=600, bbox_inches='tight', pad_inches=0.1)

# Displaying the plot
plt.show()